In [35]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import pandas as pd
import yaml
import os

with open("config.yml", 'r') as ymlfile:
    cfg = yaml.load(ymlfile)

#import data
bulkdir = "Data/Slideseq/" + cfg['default']['slideseqfolder'] + "/results/Bulk"
X = pd.read_csv(os.path.join(bulkdir, "X_bulk.csv"))
b = pd.read_csv(os.path.join(bulkdir, "b_bulk.csv"))
cell_type_names = X.columns[1:]
X = X.iloc[:,1:].to_numpy()
b = b.iloc[:,1].to_numpy()

/Users/dcable/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


In [36]:
#define the model
class Model(object):
  def __init__(self):
    self.N = len(cell_type_names)
    self.w = tf.Variable([1/self.N for i in range(self.N)])

  def __call__(self, x):
    return tf.tensordot(x[0:(self.N)],tf.abs(self.w)/tf.math.reduce_sum(tf.abs(self.w)),1)
    
  def predict_batch(self, x):
    return tf.linalg.matvec(x[:,0:(self.N)],tf.abs(self.w)/tf.math.reduce_sum(tf.abs(self.w))) 

def loss(predicted_y, target_y):
  return tf.math.reduce_sum(tf.square(tf.math.log(predicted_y) - tf.math.log(target_y)))

model = Model()

In [37]:
#create the loss function
X_var = tf.compat.v1.placeholder("float32") 
Y_var = tf.compat.v1.placeholder("float32") 
my_loss = loss(model(X_var),Y_var)
batch_loss = loss(model.predict_batch(X_var),Y_var)
optimizer = tf.train.AdamOptimizer(1e-5).minimize(my_loss)
init = tf.global_variables_initializer() 


In [38]:
#train the model
training_epochs = 500
with tf.Session() as sess: 
     
    # Initializing the Variables 
    sess.run(init) 
      
    # Iterating through all the epochs 
    for epoch in range(training_epochs): 
          
        # Feeding each data point into the optimizer using Feed Dictionary 
        for (_x, _y) in zip(X, b): 
            sess.run(optimizer, feed_dict = {X_var : _x, Y_var : _y}) 
          
        # Displaying the result after every 5 epochs 
        if (epoch + 1) % 10 == 0: 
            # Calculating the cost a every epoch 
            c = sess.run(batch_loss, feed_dict = {X_var : X, Y_var : b}) 
            print("Epoch", (epoch + 1), ": cost =", c, "w =", sess.run(model.w)) 
      
    # Storing necessary values to be used outside the Session 
    training_cost = sess.run(batch_loss, feed_dict = {X_var : X, Y_var : b}) 
    weight = sess.run(model.w) 


Epoch 10 : cost = 11685.008 w = [0.1047136  0.10232875 0.09057199 0.03389134 0.10383951 0.01406264
 0.09378739 0.05748312 0.03797298 0.07770702 0.03964967 0.00702102
 0.0032611  0.12053379 0.12671639 0.07108006 0.02295061 0.09482143
 0.04525581]
Epoch 20 : cost = 10932.791 w = [ 1.19655155e-01  1.20011739e-01  8.85331407e-02  6.72862539e-03
  1.29327506e-01 -8.47126998e-04  1.10884517e-01  3.01384050e-02
  6.14557648e-03  7.81283751e-02  3.28940689e-03  4.65672892e-05
  2.93702673e-04  1.53897688e-01  1.67656958e-01  6.90797269e-02
  6.08184200e-04  1.10063151e-01  1.86175294e-02]
Epoch 30 : cost = 10782.589 w = [ 1.23021282e-01  1.27888843e-01  7.43584409e-02 -4.48991312e-04
  1.43384710e-01  1.23399915e-03  1.18084431e-01  4.13066661e-03
 -1.72170345e-04  7.25509599e-02  2.50306184e-04  3.70631569e-05
  2.78649677e-04  1.75679922e-01  1.97126284e-01  6.37561083e-02
  9.70406982e-04  1.16375774e-01  3.27669736e-03]
Epoch 40 : cost = 10751.25 w = [ 1.23968594e-01  1.33227348e-01  5.757

KeyboardInterrupt: 

In [21]:
weight_df = pd.DataFrame(abs(weight),index=cell_type_names,columns=["Weight"])
weight_df.to_csv(os.path.join(bulkdir, "weights.csv"))


In [52]:
import numpy as np
weight = [ 1.23283662e-01,  1.73089460e-01 , 4.19172022e-04, -5.16497239e-04,
  2.14016780e-01  ,1.77152443e-03,  1.48490205e-01 ,-2.27184122e-04,
 -1.04071310e-04  ,4.14524488e-02 , 2.91331118e-04 ,-9.21871178e-05,
  2.81929038e-04 , 3.41065317e-01  ,4.57597822e-01,  6.14649951e-02,
  1.71664346e-03,  1.21774025e-01  ,2.70181685e-03]
np.array(weight) / sum(weight)

array([ 7.30147038e-02,  1.02512169e-01,  2.48254477e-04, -3.05895301e-04,
        1.26751360e-01,  1.04918470e-03,  8.79432697e-02, -1.34549713e-04,
       -6.16361953e-05,  2.45501977e-02,  1.72540748e-04, -5.45977868e-05,
        1.66972370e-04,  2.01995809e-01,  2.71012143e-01,  3.64026209e-02,
        1.01668146e-03,  7.21206218e-02,  1.60015004e-03])